In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Explore public HUD REAC data

Author: Jack Vandeleuv

Data files:
* [Public Housing Physical Inspection Scores (2016-2021)](https://www.huduser.gov/portal/datasets/pis.html#2021_data-collapse)
* [Multifamily Physical Inspection Scores (2016-2021)](https://www.huduser.gov/portal/datasets/pis.html#2021_data-collapse)

This notebook will explore the publicly released scores from the U.S. Department of Housing and Urban Development's Real Estate Assessment Center (HUD REAC). These scores are derived from physical inspections on HUD properties.

The goal of this notebook is to aggregate REAC data on a city-by-city basis, so we'll focus on data relevant to that analysis.

# Loading and cleaning

In [3]:
seed = 538

Place the data files in the data directory and load into pandas.

In [4]:
WORKING_DIR = '../data/'
MULTIFAMILY_FILES = [
    'multifamily_physical_inspection_scores_0321.xlsx',
    'multifamily_physical_inspection_scores_0620.xlsx',
    'multifamily-physical-inspection-scores-2016.xlsx',
    'multifamily-physical-inspection-scores-2018.xlsx',
    'multifamily-physical-inspection-scores-2019.xlsx'
]

PUBLIC_HOUSING_FILES = [
    'public_housing_physical_inspection_scores_0321.xlsx',
    'public_housing_physical_inspection_scores_0620.xlsx',
    'public-housing-physical-inspection-scores-2016.xlsx',
    'public-housing-physical-inspection-scores-2018.xlsx',
    'public-housing-physical-inspection-scores-2019.xlsx'
]

The zipcode columns is alternately named ZIP and ZIPCODE. We'll rename all columns labeled ZIPCODE as ZIP for consistency sake.

In [5]:
multi_dfs = []
public_dfs = []

for i, public_file in enumerate(PUBLIC_HOUSING_FILES):
    public_df = pd.read_excel(WORKING_DIR + public_file)
    multi_df = pd.read_excel(WORKING_DIR + MULTIFAMILY_FILES[i])

    public_df = public_df.rename(columns={'ZIPCODE': 'ZIP'}, )
    multi_df = multi_df.rename(columns={'ZIPCODE': 'ZIP'})

    public_dfs.append(public_df)
    multi_dfs.append(multi_df)

multi = pd.concat(multi_dfs, axis=0)
public = pd.concat(public_dfs, axis=0)

Because the REAC dataset shows the latest inspection for each property, and we're merging data from multiple years, there will be a significant number of duplicates. We'll drop these. We'll define a duplicate as two inspections with the same INSPECTION_ID.

In [6]:
multi = multi[~multi.INSPECTION_ID.duplicated()]
public = public[~public.INSPECTION_ID.duplicated()]

In [7]:
multi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63807 entries, 0 to 27876
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   INSPECTION_ID     63807 non-null  int64  
 1   PROPERTY_ID       63807 non-null  int64  
 2   PROPERTY_NAME     63807 non-null  object 
 3   ADDRESS           62624 non-null  object 
 4   CITY              63807 non-null  object 
 5   CBSA_NAME         58603 non-null  object 
 6   CBSA_CODE         63777 non-null  float64
 7   COUNTY_NAME       63780 non-null  object 
 8   COUNTY_CODE       63781 non-null  float64
 9   STATE_NAME        52362 non-null  object 
 10  STATE_CODE        63807 non-null  object 
 11  ZIP               63787 non-null  float64
 12  LATITUDE          63781 non-null  float64
 13  LONGITUDE         63781 non-null  float64
 14  LOCATION_QUALITY  63781 non-null  object 
 15  INSPECTION_SCORE  63807 non-null  int64  
 16  INSPECTION_DATE   63807 non-null  object

In [8]:
public.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16962 entries, 0 to 6778
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   INSPECTION_ID     16962 non-null  int64  
 1   DEVELOPMENT_ID    16962 non-null  object 
 2   DEVELOPMENT_NAME  16962 non-null  object 
 3   ADDRESS           6639 non-null   object 
 4   CITY              16962 non-null  object 
 5   CBSA_NAME         14634 non-null  object 
 6   CBSA_CODE         16956 non-null  float64
 7   COUNTY_NAME       16956 non-null  object 
 8   COUNTY_CODE       16956 non-null  float64
 9   STATE_NAME        13420 non-null  object 
 10  STATE_CODE        16958 non-null  object 
 11  ZIP               16960 non-null  float64
 12  LATITUDE          16956 non-null  float64
 13  LONGITUDE         16956 non-null  float64
 14  LOCATION_QUALITY  16960 non-null  object 
 15  PHA_CODE          16962 non-null  object 
 16  PHA_NAME          16962 non-null  object 

Let's also look at STATE_NAME and STATE_CODE, which has a lot of null values.

In [9]:
multi.sample(n=5, random_state=seed).loc[:, ['STATE_NAME', 'STATE_CODE']]

,STATE_NAME,STATE_CODE
7548,WV,54
16258,GA,13
3573,VA,51
22460,NaN,MA
4033,AL,1


In [10]:
public.sample(n=5, random_state=seed).loc[:, ['STATE_NAME', 'STATE_CODE']]

,STATE_NAME,STATE_CODE
4103,MD,24
3385,NaN,IN
615,AL,1
3059,VA,51.0
6652,NaN,VA


We also have two different state code formats. We'll combine these into a single column with only the two-letter state codes.

In [11]:
def combine_columns(row):
    if pd.isnull(row['STATE_NAME']):
        return row['STATE_CODE']
    else:
        return row['STATE_NAME']

public['STATE'] = public.apply(combine_columns, axis=1)
multi['STATE'] = multi.apply(combine_columns, axis=1)

public = public.drop(columns=['STATE_NAME', 'STATE_CODE'])
multi = multi.drop(columns=['STATE_NAME', 'STATE_CODE'])

Let's look again to see what nulls are left.

In [12]:
public.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16962 entries, 0 to 6778
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   INSPECTION_ID     16962 non-null  int64  
 1   DEVELOPMENT_ID    16962 non-null  object 
 2   DEVELOPMENT_NAME  16962 non-null  object 
 3   ADDRESS           6639 non-null   object 
 4   CITY              16962 non-null  object 
 5   CBSA_NAME         14634 non-null  object 
 6   CBSA_CODE         16956 non-null  float64
 7   COUNTY_NAME       16956 non-null  object 
 8   COUNTY_CODE       16956 non-null  float64
 9   ZIP               16960 non-null  float64
 10  LATITUDE          16956 non-null  float64
 11  LONGITUDE         16956 non-null  float64
 12  LOCATION_QUALITY  16960 non-null  object 
 13  PHA_CODE          16962 non-null  object 
 14  PHA_NAME          16962 non-null  object 
 15  INSPECTION_SCORE  16962 non-null  int64  
 16  INSPECTION_DATE   16962 non-null  object 

In [13]:
multi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63807 entries, 0 to 27876
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   INSPECTION_ID     63807 non-null  int64  
 1   PROPERTY_ID       63807 non-null  int64  
 2   PROPERTY_NAME     63807 non-null  object 
 3   ADDRESS           62624 non-null  object 
 4   CITY              63807 non-null  object 
 5   CBSA_NAME         58603 non-null  object 
 6   CBSA_CODE         63777 non-null  float64
 7   COUNTY_NAME       63780 non-null  object 
 8   COUNTY_CODE       63781 non-null  float64
 9   ZIP               63787 non-null  float64
 10  LATITUDE          63781 non-null  float64
 11  LONGITUDE         63781 non-null  float64
 12  LOCATION_QUALITY  63781 non-null  object 
 13  INSPECTION_SCORE  63807 non-null  int64  
 14  INSPECTION_DATE   63807 non-null  object 
 15  FIPS_STATE_CODE   11445 non-null  float64
 16  STATE             63807 non-null  object

We have no nulls in CITY and STATE. Let's look at the remaining nulls in ZIP.

In [14]:
print(len(public[public.ZIP.isna()]))
print('Unique states represented among null zipcodes:', public[public.ZIP.isna()].STATE.unique())

2
Unique states represented among null zipcodes: ['PR']


In [15]:
print(len(multi[multi.ZIP.isna()]))
print('Unique states represented among null zipcodes:', multi[multi.ZIP.isna()].STATE.unique())

20
Unique states represented among null zipcodes: ['MA' 'PR' 'NJ' 'VA' 'GA' 'WY' 'TN' 'OH' 'CA' 'DC' 'UT']


Standardize the city names to be upper-case.

In [16]:
multi.CITY = multi.CITY.str.upper()
public.CITY = public.CITY.str.upper()

We have a small number of inspection without zipcode data. We'll leave these for now, and see if we can perform the spatial join without the zipcode data.

We'll convert the datetimes to a standard format.

In [17]:
public['INSPECTION_DATE'] = pd.to_datetime(public['INSPECTION_DATE'], infer_datetime_format=True)
multi['INSPECTION_DATE'] = pd.to_datetime(multi['INSPECTION_DATE'], infer_datetime_format=True)

Let's check to see if we have any cities in the same state with the same name. We'll check this using latitude and longitude.

In [18]:
multi_citystate = multi.loc[:, ['CITY', 'STATE']]
multi_unique_locs = multi_citystate[~multi_citystate.duplicated()].values.tolist()

Let's look at this distribution of scores by date range.

In [23]:
multi.groupby(by=multi.INSPECTION_DATE.dt.year).size()

INSPECTION_DATE
2013     2949
2014     8542
2015    10762
2016     9104
2017     7471
2018    11856
2019    11286
2020     1835
2021        2
dtype: int64

In [26]:
public.groupby(by=public.INSPECTION_DATE.dt.year).size()

INSPECTION_DATE
2005       1
2012       5
2013     336
2014    1739
2015    3272
2016    2424
2017    3302
2018    2438
2019    2566
2020     879
dtype: int64

The bulk of inspections in the dataset occurred between 2013 and 2020, with a drop-off when the 2019 pandemic started. 

### Note: there are a few values that have mismatch between coordinates and the listed address, or that are an outlier in terms of min/max latitude/longitude. We can remove these, although there are likely not too many values.

In general, a spread-out city like KCMO might vary by about 1 latitude point from top to bottom.

In [195]:
max_lats = []
max_longs = []

for city, state in multi_unique_locs:
    city_mask = multi.CITY == city
    state_mask = multi.STATE == state
    multi_sub = multi[city_mask & state_mask]
    max_lats.append((city, state, multi_sub.LATITUDE.max() - multi_sub.LATITUDE.min()))
    max_longs.append((city, state, multi_sub.LONGITUDE.max() - multi_sub.LONGITUDE.min()))

In [199]:
sorted(max_lats, key=lambda x: x[2], reverse=True)[:10]

[('WINCHESTER', 'VA', 1.9351839999999996),
 ('HAMILTON TOWNSHIP', 'NJ', 0.8900429999999986),
 ('REDLANDS', 'CA', 0.8825670000000017),
 ('SAINT THOMAS', 'VI', 0.6152999999999977),
 ('NASHVILLE', 'TN', 0.45262400000000014),
 ('HOUSTON', 'TX', 0.4393570000000011),
 ('FAITH', 'SD', 0.43188200000000165),
 ('SAN DIEGO', 'CA', 0.40525399999999934),
 ('KANSAS CITY', 'MO', 0.40067200000000014),
 ('MIAMI', 'FL', 0.40031199999999956)]

The top value has an incorrect latitude and longitude, but the address is valid.

In [191]:
city_mask = multi.CITY == 'WINCHESTER'
state_mask = multi.STATE == 'VA'

multi[city_mask & state_mask].sort_values(by='LATITUDE')

,INSPECTION_ID,PROPERTY_ID,PROPERTY_NAME,ADDRESS,CITY,CBSA_NAME,CBSA_CODE,COUNTY_NAME,COUNTY_CODE,ZIP,LATITUDE,LONGITUDE,LOCATION_QUALITY,INSPECTION_SCORE,INSPECTION_DATE,FIPS_STATE_CODE,STATE
7934,597035,800225812,Spring Arbor5-Winchester,Echo Lane and Northwestern Pike,WINCHESTER,"Blacksburg-Christiansburg, VA Metropolitan Sta...",13980.0,Montgomery,121.0,24060.0,37.248300,-80.462400,T,84,2018-10-09,NaN,VA
8153,598681,800061972,THE WILLOWS AT MEADOW BRANCH,1881 Harvest Dr,WINCHESTER,"Winchester, VA-WV Metropolitan Statistical Area",49020.0,Winchester,840.0,22601.0,39.169797,-78.185480,R,84,2019-03-08,NaN,VA
14617,542708,800061972,THE WILLOWS AT MEADOW BRANCH,1881 Harvest Dr,WINCHESTER,"Winchester, VA-WV Metropolitan Statistical Area",49020.0,Winchester,840.0,22601.0,39.169813,-78.185438,R,83,2015-04-09,NaN,VA
23302,594561,800061972,THE WILLOWS AT MEADOW BRANCH,1881 Harvest Dr,WINCHESTER,"Winchester, VA-WV Metropolitan Statistical Area",49020.0,Winchester,840.0,22601.0,39.169813,-78.185438,R,81,2017-03-16,51.0,VA
10538,612725,800022156,WINCHESTER HOUSE,27 S Cameron St,WINCHESTER,"Winchester, VA-WV Metropolitan Statistical Area",49020.0,Winchester,840.0,22601.0,39.183454,-78.164480,R,87,2018-02-14,NaN,VA
21961,558503,800022156,WINCHESTER HOUSE,27 S Cameron St,WINCHESTER,"Winchester, VA-WV Metropolitan Statistical Area",49020.0,Winchester,840.0,22601.0,39.183484,-78.164469,R,88,2015-10-19,NaN,VA


The top value is miscoded as being in Hamilton Township when it's not, which results in a high latitude rating

In [192]:
city_mask = multi.CITY == 'HAMILTON TOWNSHIP'
state_mask = multi.STATE == 'NJ'

multi[city_mask & state_mask].sort_values(by='LATITUDE')

,INSPECTION_ID,PROPERTY_ID,PROPERTY_NAME,ADDRESS,CITY,CBSA_NAME,CBSA_CODE,COUNTY_NAME,COUNTY_CODE,ZIP,LATITUDE,LONGITUDE,LOCATION_QUALITY,INSPECTION_SCORE,INSPECTION_DATE,FIPS_STATE_CODE,STATE
4433,585235,800059453,HARDINGS RUN SUPERVISED APTS.,4433 Yorktown Place,HAMILTON TOWNSHIP,"Atlantic City-Hammonton, NJ Metropolitan Stati...",12100.0,Atlantic,1.0,8244.0,39.322400,-74.589300,T,96,2016-12-13,NaN,NJ
3678,514237,800014163,Hamilton Supervised Apartments,196 Route 156,HAMILTON TOWNSHIP,"Trenton, NJ Metropolitan Statistical Area",45940.0,Mercer,21.0,8620.0,40.180286,-74.667300,R,94,2013-12-19,NaN,NJ
17169,633139,800014163,Hamilton Supervised Apartments,196 Route 156,HAMILTON TOWNSHIP,"Trenton-Princeton, NJ Metropolitan Statistical...",45940.0,Mercer,21.0,8620.0,40.180292,-74.667289,R,82,2018-12-06,NaN,NJ
19126,584612,800014163,Hamilton Supervised Apartments,196 Route 156,HAMILTON TOWNSHIP,"Trenton, NJ Metropolitan Statistical Area",45940.0,Mercer,21.0,8620.0,40.180292,-74.667289,R,85,2016-12-16,34.0,NJ
17163,633129,800014161,HAMILTON GROUP HOME,1330 Cedar Ln,HAMILTON TOWNSHIP,"Trenton-Princeton, NJ Metropolitan Statistical...",45940.0,Mercer,21.0,8610.0,40.212431,-74.724466,R,59,2018-12-10,NaN,NJ
3671,514211,800014161,HAMILTON GROUP HOME,1330 Cedar Ln,HAMILTON TOWNSHIP,"Trenton, NJ Metropolitan Statistical Area",45940.0,Mercer,21.0,8610.0,40.212443,-74.724456,R,99,2013-12-19,NaN,NJ
19152,584647,800014161,HAMILTON GROUP HOME,1330 Cedar Ln,HAMILTON TOWNSHIP,"Trenton, NJ Metropolitan Statistical Area",45940.0,Mercer,21.0,8610.0,40.212443,-74.724456,R,85,2016-12-15,34.0,NJ


In [197]:
sorted(max_longs, key=lambda x: x[2], reverse=True)[:5]

[('TAMUNING', 'GU', 289.606548),
 ('WINCHESTER', 'VA', 2.2979310000000055),
 ('FAITH', 'SD', 0.6390240000000063),
 ('BARRANQUITAS', 'PR', 0.6384999999999934),
 ('HOUSTON', 'TX', 0.5910080000000022)]